In [1]:
import os
import numpy as np
import pandas as pd
from PIL import Image
from time import time as timer
import matplotlib.pyplot as plt

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.utils.data import random_split, Dataset, DataLoader

In [2]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/cifar-10/trainLabels.csv
/kaggle/input/cifar-10/sampleSubmission.csv
/kaggle/input/cifar-10/test.7z
/kaggle/input/cifar-10/train.7z


In [18]:
labels_csv = pd.read_csv("/kaggle/input/cifar-10/trainLabels.csv")
labels_csv

,id,label
0,1,frog
1,2,truck
2,3,truck
3,4,deer
4,5,automobile
...,...,...
49995,49996,bird
49996,49997,frog
49997,49998,truck
49998,49999,automobile


In [19]:
label_mapping = {label: idx for idx, label in enumerate(labels_csv['label'].unique())}

labels_csv.rename({"label": "label_txt"}, axis=1, inplace=True)
labels_csv['label'] = labels_csv['label_txt'].map(label_mapping)

print(label_mapping, labels_csv)

{'frog': 0, 'truck': 1, 'deer': 2, 'automobile': 3, 'bird': 4, 'horse': 5, 'ship': 6, 'cat': 7, 'dog': 8, 'airplane': 9}           id   label_txt  label
0          1        frog      0
1          2       truck      1
2          3       truck      1
3          4        deer      2
4          5  automobile      3
...      ...         ...    ...
49995  49996        bird      4
49996  49997        frog      0
49997  49998       truck      1
49998  49999  automobile      3
49999  50000  automobile      3

[50000 rows x 3 columns]


In [ ]:
!pip install py7zr

import py7zr

archive_path = '/kaggle/input/cifar-10/train.7z'

extract_to = '/kaggle/working/'

with py7zr.SevenZipFile(archive_path, mode='r') as z:
    z.extractall(path=extract_to)

print(f"Архив распакован в {extract_to}")

In [20]:
train_dir = "/kaggle/working/train"
# test_dir = "/kaggle/working/test"

train_imgs = os.listdir(train_dir)

img1 = Image.open(train_dir + "/" + train_imgs[0])
img1.size

(32, 32)

In [21]:
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.2),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,)),
])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

In [23]:
class CustomDataset(Dataset):
    def __init__(self, images_dir, labels_csv, transform=None):
        self.transforms = transform
        self.imgs_dir = images_dir
        self.labels_csv = labels_csv

    def __len__(self):
        return len(self.labels_csv)
    
    def __getitem__(self, idx):
        img_name = str(self.labels_csv.iloc[idx, 0]) + ".png"
        label = self.labels_csv.loc[idx, "label"]

        img_path = os.path.join(self.imgs_dir, img_name)

        img = Image.open(img_path).convert("RGB")

        if self.transforms:
            img = self.transforms(img)
        
        return img, label

In [24]:
train_dataset = CustomDataset(
    images_dir = "/kaggle/working/train", labels_csv = labels_csv,
    transform = transform
)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [35]:
from tqdm import tqdm

class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.flatten = nn.Flatten()
        self.layer1 = nn.Linear(32 * 32 * 3, 64)
        self.layer2 = nn.Linear(64, 64)
        self.layer3 = nn.Linear(64, 10)
        self.sigmoid = nn.Sigmoid()
        self.relu = nn.ReLU()
    
    def forward(self, x):
        x = self.flatten(x)
        x = self.sigmoid(self.layer1(x))
        x = self.sigmoid(self.layer2(x))
        x = self.layer3(x)
        return x


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Model().to(device)

cr = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

corrects = 0
epochs = 10

for i in range(epochs):
    cost = 0
    for image, answer in tqdm(train_loader):
        image = image.to(device)
        answer = answer.to(device)

        optimizer.zero_grad()
        predictions = model(image)
        error = cr(predictions, answer)
        error.backward()
        optimizer.step()
        cost += error.item()

        if i == epochs - 1:
            corrects += (predictions.argmax(1) == answer).sum().item()

    print(f"Epoch {i+1}, Loss: {cost / len(train_loader):.4f}")

total_samples = len(train_loader.dataset)
print("Accuracy:", corrects, "/", total_samples, "=", corrects / total_samples)

100%|██████████| 1563/1563 [00:32<00:00, 48.22it/s]


Epoch 1, Loss: 1.8680


100%|██████████| 1563/1563 [00:31<00:00, 48.92it/s]


Epoch 2, Loss: 1.7767


100%|██████████| 1563/1563 [00:31<00:00, 49.36it/s]


...

Epoch 8, Loss: 1.6849


100%|██████████| 1563/1563 [00:32<00:00, 48.79it/s]


Epoch 9, Loss: 1.6840


100%|██████████| 1563/1563 [00:32<00:00, 48.33it/s]

Epoch 10, Loss: 1.6741
Accuracy: 20082 / 50000 = 0.40164


In [ ]:
train_split = int(0.8 * len(train_dataset))
val_split = int(0.2 * len(train_dataset))
train_split, val_split

train_dataset, val_dataset = random_split(train_dataset, [train_split, val_split])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [39]:
!pip install py7zr

import py7zr

archive_path = '/kaggle/input/cifar-10/test.7z'

extract_to = '/kaggle/working/'

with py7zr.SevenZipFile(archive_path, mode='r') as z:
    z.extractall(path=extract_to)

print(f"Архив распакован в {extract_to}")

Архив распакован в /kaggle/working/


In [40]:
class CustomTestDataset(Dataset):
    def __init__(self, images_dir, transform=None):
        self.images_dir = images_dir
        self.transform = transform
        self.img_files = sorted(os.listdir(images_dir))

    def __len__(self):
        return len(self.img_files)

    def __getitem__(self, idx):
        img_path = os.path.join(self.images_dir, self.img_files[idx])
        image = Image.open(img_path).convert("RGB")

        if self.transform:
            image = self.transform(image)

        return image, self.img_files[idx]


test_dataset = CustomTestDataset("/kaggle/working/test", test_transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [41]:
model.eval()

Model(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (layer1): Linear(in_features=3072, out_features=64, bias=True)
  (layer2): Linear(in_features=64, out_features=64, bias=True)
  (layer3): Linear(in_features=64, out_features=10, bias=True)
  (sigmoid): Sigmoid()
  (relu): ReLU()
)

In [42]:
all_preds = []
img_idxs = []

with torch.no_grad():
    for images, idxs in tqdm(test_loader):
        images = images.to(device)

        outputs = model(images)

        _, preds = torch.max(outputs, 1)

        all_preds.extend(preds.cpu().numpy())
        img_idxs.extend(idxs)

100%|██████████| 9375/9375 [02:32<00:00, 61.63it/s]


In [43]:
idxs_to_classes = {v: k for k, v in label_mapping.items()}

predicted_labels = [idxs_to_classes[pred] for pred in all_preds]

submission_df = pd.DataFrame({
    "id": [int(file.split(".")[0]) for file in img_idxs],  
    "label": predicted_labels
})

submission_df = submission_df.sort_values(by="id")

submission_df.to_csv("submission.csv", index=False)

In [44]:
submission_df

,id,label
0,1,dog
111111,2,airplane
222222,3,horse
233334,4,airplane
244445,5,airplane
...,...,...
222218,299996,dog
222219,299997,deer
222220,299998,cat
222221,299999,bird
